# Advent of Code - 2024 - Day 6 - Problem 2

https://adventofcode.com/2024/day/6

## Load Source Data

Load the map data into `DATA`.

In [1]:
f = open("data/day6.txt", "r")
DATA = list(map(str.strip, f.readlines()))
f.close()

## Create Maze Class

In [ ]:
class Maze:

    def __init__(self, data):
        self.data = list(map(list, data))

    def is_valid(self, location):
        row, col = location
        if row < 0 or row >= len(self.data):      return False
        if col < 0 or col >= len(self.data[row]): return False
        return True

    def get_value(self, location):
        row, col = location
        return self.data[row][col]

    def find_value(self, value):
        for row in range(len(self.data)):
            for col in range(len(self.data[row])):
                if self.data[row][col] == value:
                    return (row, col)
        return None

## Create Cursor Class

In [3]:
ORIENTATION_UP = 0
ORIENTATION_RIGHT = 1
ORIENTATION_DOWN = 2
ORIENTATION_LEFT = 3


class Cursor:

    def __init__(self, location, orientation):
        self.location = location
        self.orientation = orientation

    def __str__(self):
        return f"({self.location[0]},{self.location[1]},{self.orientation})"

    # STATIC
    def get_next_location(self, location, orientation):

        row, col = location

        if orientation == ORIENTATION_UP:      return (row - 1, col)
        elif orientation == ORIENTATION_RIGHT: return (row, col + 1)
        elif orientation == ORIENTATION_DOWN:  return (row + 1, col)
        else:                                  return (row, col - 1)  # LEFT

    def get_next_locations(self):
        location = self.location
        orientation = self.orientation
        for idx in range(4):
            yield Cursor(self.get_next_location(location, orientation), orientation)
            orientation = (orientation + 1) % 4

## Create Guard Class

In [4]:
GUARD_UP = "^"


class MoveFailure(Exception):
    pass


class Guard:

    def __init__(self, maze):
        self._maze = maze
        self._cursor = None
        self._locations = set()  # locations in guard's path
        self._cursor_strings = set()  # string-ified cursors for guard's path
        self._obstructed_location = None
        self.possible_obstructions = set()

    def _clone(self, obstructed_location):
        g = Guard(self._maze)
        g._cursor = self._cursor
        g._locations = set(self._locations)
        g._cursor_strings = set(self._cursor_strings)
        g._obstructed_location = obstructed_location
        return g

    def set_cursor(self, cursor):
        self._cursor = cursor
        self._locations.add(cursor.location)
        self._cursor_strings.add(str(cursor))

    def _is_obstructed(self, location):
        if self._maze.get_value(location) == "#":
            return True
        if self._obstructed_location == location:
            return True
        return False

    def _move(self, escaping):

        for next_cursor in self._cursor.get_next_locations():

            # See if we've moved off the map.
            #
            if not self._maze.is_valid(next_cursor.location):
                return True

            # Move if the direction isn't blocked.
            #
            if not self._is_obstructed(next_cursor.location):

                if not escaping and next_cursor.location not in self._locations:
                    if not self._can_escape(next_cursor.location):
                        self.possible_obstructions.add(next_cursor.location)

                if str(next_cursor) in self._cursor_strings:
                    raise MoveFailure("Loop detected.")

                self._cursor = next_cursor
                self._locations.add(self._cursor.location)
                self._cursor_strings.add(str(self._cursor))

                return False

        raise Exception("No possible moves.")

    def _can_escape(self, obstructed_location):
        g = self._clone(obstructed_location)
        try:
            g._escape()
            return True
        except MoveFailure:
            return False

    def _escape(self):
        while not self._move(True):
            pass

    def search(self):
        while not self._move(False):
            pass

## Move Guard Around Maze

In [5]:
maze = Maze(DATA)

location = maze.find_value(GUARD_UP)
cursor = Cursor(location, ORIENTATION_UP)

guard = Guard(maze)
guard.set_cursor(cursor)
guard.search()
print(f"obstruction_count = {len(guard.possible_obstructions)}")

obstruction_count = 1705
